<a href="https://colab.research.google.com/github/amrutadeo-22/resnet/blob/main/hybrid_sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install certifi


In [2]:
import ssl
import certifi

ssl._create_default_https_context = ssl._create_unverified_context


In [ ]:
import ssl
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import EuroSAT

# Disable SSL verification (not recommended for production)
ssl._create_default_https_context = ssl._create_unverified_context

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = nn.ReLU(inplace=True)(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = nn.ReLU(inplace=True)(out)
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, shortcut_type='B'):
        super(Bottleneck, self).__init__()
        self.stride = stride

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

        if shortcut_type == 'C' or (shortcut_type == 'B' and in_channels != out_channels * self.expansion):
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )
        elif in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.AvgPool2d(1, stride=stride),
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        out = nn.ReLU(inplace=True)(self.bn1(self.conv1(x)))
        out = nn.ReLU(inplace=True)(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = nn.ReLU(inplace=True)(out)
        return out

class HybridResNet(nn.Module):
    def __init__(self, num_classes=10, input_size=64):
        super(HybridResNet, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(BasicBlock, 16, 2, stride=1)
        self.layer2 = self._make_layer(Bottleneck, 32, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 64, 2, stride=2)
        self.layer4 = self._make_layer(Bottleneck, 128, 2, stride=2)

        # Calculate the size of the output of the last conv layer
        self._initialize_output_size(input_size)

        self.linear = nn.Linear(self.conv_output_size, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def _initialize_output_size(self, input_size):
        with torch.no_grad():
            x = torch.zeros(1, 3, input_size, input_size)
            x = self.conv1(x)
            x = self.layer1(x)
            x = self.layer2(x)
            x = self.layer3(x)
            x = self.layer4(x)
            x = nn.AvgPool2d(4)(x)
            self.conv_output_size = x.view(1, -1).size(1)

    def forward(self, x):
        out = nn.ReLU(inplace=True)(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = nn.AvgPool2d(4)(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def train(epoch, net, trainloader, optimizer, criterion, device):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 10 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(inputs)}/{len(trainloader.dataset)}] '
                  f'Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}%')

def test(epoch, net, testloader, criterion, device):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Epoch: {epoch} | Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}%')


def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    transform_train = transforms.Compose([
        transforms.RandomCrop(64, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.3444, 0.3809, 0.3432), (0.1915, 0.1594, 0.1403)),
    ])

    transform_test = transforms.Compose([
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize((0.3444, 0.3809, 0.3432), (0.1915, 0.1594, 0.1403)),
    ])

    trainset = EuroSAT(root='./data', download=True, transform=transform_train)
    trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

    testset = EuroSAT(root='./data', download=True, transform=transform_test)
    testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

    net = HybridResNet(num_classes=10, input_size=64).to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

    for epoch in range(0, 20):
        train(epoch, net, trainloader, optimizer, criterion, device)
        test(epoch, net, testloader, criterion, device)
        scheduler.step()

if __name__ == '__main__':
    main()


100%|██████████| 94280567/94280567 [00:03<00:00, 25294960.87it/s]


Extracting ./data/eurosat/EuroSAT.zip to ./data/eurosat


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Train Epoch: 0 [0/27000] Loss: 2.343 | Acc: 15.625%
Train Epoch: 0 [1280/27000] Loss: 14.546 | Acc: 12.926%
Train Epoch: 0 [2560/27000] Loss: 9.572 | Acc: 13.430%
Train Epoch: 0 [3840/27000] Loss: 7.340 | Acc: 14.642%
Train Epoch: 0 [5120/27000] Loss: 6.201 | Acc: 14.634%
Train Epoch: 0 [6400/27000] Loss: 5.431 | Acc: 15.058%
Train Epoch: 0 [7680/27000] Loss: 4.912 | Acc: 15.407%
Train Epoch: 0 [8960/27000] Loss: 4.744 | Acc: 15.328%
Train Epoch: 0 [10240/27000] Loss: 4.535 | Acc: 15.258%
Train Epoch: 0 [11520/27000] Loss: 4.426 | Acc: 15.290%
Train Epoch: 0 [12800/27000] Loss: 4.229 | Acc: 15.540%
Train Epoch: 0 [14080/27000] Loss: 4.051 | Acc: 15.885%
Train Epoch: 0 [15360/27000] Loss: 3.918 | Acc: 16.581%
Train Epoch: 0 [16640/27000] Loss: 3.780 | Acc: 17.110%
Train Epoch: 0 [17920/27000] Loss: 3.650 | Acc: 17.814%
Train Epoch: 0 [19200/27000] Loss: 3.549 | Acc: 18.409%
Train Epoch: 0 [20480/27000] Loss: 3.446 | Acc: 18.973%
Train Epoch: 0 [21760/27000] Loss: 3.357 | Acc: 19.243%
Tr